In [93]:
import numpy as np
import pandas as pd

In [94]:
series1 = pd.Series([1,4,5,10,9,3,2,6,8,4])
series2 = pd.Series([1,7,3,4,1,10,5,4,7,4])

In [95]:
def fill_dtw_cost_matrix(s1, s2):
    l_s_1, l_s_2 = len(s1), len(s2)
    
    cost_matrix = np.zeros((l_s_1+1, l_s_2+1))

    for i in range(l_s_1+1):
        for j in range(l_s_2+1):
            cost_matrix[i, j] = np.inf

    cost_matrix[0, 0] = 0
    
    for i in range(1, l_s_1+1):
        for j in range(1, l_s_2+1):
            cost = abs(s1[i-1] - s2[j-1])
            #take last min from the window
            prev_min = np.min([cost_matrix[i-1, j], cost_matrix[i, j-1], cost_matrix[i-1, j-1]])
            cost_matrix[i, j] = cost + prev_min

    return cost_matrix

In [96]:
dtw_cost_matrix = fill_dtw_cost_matrix(series1,series2)

In [97]:
dtw_cost_matrix

array([[ 0., inf, inf, inf, inf, inf, inf, inf, inf, inf, inf],
       [inf,  0.,  6.,  8., 11., 11., 20., 24., 27., 33., 36.],
       [inf,  3.,  3.,  4.,  4.,  7., 13., 14., 14., 17., 17.],
       [inf,  7.,  5.,  5.,  5.,  8., 12., 12., 13., 15., 16.],
       [inf, 16.,  8., 12., 11., 14.,  8., 13., 18., 16., 21.],
       [inf, 24., 10., 14., 16., 19.,  9., 12., 17., 18., 21.],
       [inf, 26., 14., 10., 11., 13., 16., 11., 12., 16., 17.],
       [inf, 27., 19., 11., 12., 12., 20., 14., 13., 17., 18.],
       [inf, 32., 20., 14., 13., 17., 16., 15., 15., 14., 16.],
       [inf, 39., 21., 19., 17., 20., 18., 18., 19., 15., 18.],
       [inf, 42., 24., 20., 17., 20., 24., 19., 18., 18., 15.]])

In [114]:
def get_dtw_path(mat):

    mat_path = np.full_like(mat, np.NaN)

    i, j = mat.shape[0]-1, mat.shape[1]-1
    curr_min = mat[i-1,j-1]
    #print[curr_min]
    #print[i,j]
    out=0
    mat_path[i, j] = mat[i, j]
    while curr_min > 0:
        out+=1
        
        if mat[i-1, j-1] <= mat[i, j]:
            mat_path[i-1, j-1] = mat[i-1, j-1]
            i -= 1
            j -= 1
            continue
            #print('A')
        if mat[i, j-1] <= mat[i, j]:
            mat_path[i, j-1] = mat[i, j-1]
            j -= 1
            continue
            #print('B')
        if mat[i-1, j] <= mat[i, j]:
            mat_path[i-1, j] = mat[i-1, j]
            i -= 1
            continue
            #print('C')
        
        #print(curr_min)
        #print(i,j)

        if out > 30:
            break

    return mat_path

In [115]:
path = get_dtw_path(dtw_cost_matrix)
dtw_cost_matrix, path

(array([[ 0., inf, inf, inf, inf, inf, inf, inf, inf, inf, inf],
        [inf,  0.,  6.,  8., 11., 11., 20., 24., 27., 33., 36.],
        [inf,  3.,  3.,  4.,  4.,  7., 13., 14., 14., 17., 17.],
        [inf,  7.,  5.,  5.,  5.,  8., 12., 12., 13., 15., 16.],
        [inf, 16.,  8., 12., 11., 14.,  8., 13., 18., 16., 21.],
        [inf, 24., 10., 14., 16., 19.,  9., 12., 17., 18., 21.],
        [inf, 26., 14., 10., 11., 13., 16., 11., 12., 16., 17.],
        [inf, 27., 19., 11., 12., 12., 20., 14., 13., 17., 18.],
        [inf, 32., 20., 14., 13., 17., 16., 15., 15., 14., 16.],
        [inf, 39., 21., 19., 17., 20., 18., 18., 19., 15., 18.],
        [inf, 42., 24., 20., 17., 20., 24., 19., 18., 18., 15.]]),
 array([[ 0., nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan,  0., nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan,  3.,  4.,  4., nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan,  8., nan, nan, nan, nan, nan],
        [nan, nan, nan,

In [116]:
path_count = np.count_nonzero(~np.isnan(path))
path_sum = np.nansum(path)
distance = path_sum / path_count

print(f"Pasos: {path_count}\nSuma: {path_sum}\nDistancia: {distance:.3f}")

Pasos: 13
Suma: 106.0
Distancia: 8.154


In [117]:
import numpy as np
from scipy.spatial.distance import euclidean

from fastdtw import fastdtw

s1 = np.array(series1).reshape(-1, 1)
s2 = np.array(series2).reshape(-1, 1)

distance, path = fastdtw(s1, s2, dist=euclidean)
print(distance, path, distance / len(path))

15.0 [(0, 0), (1, 1), (1, 2), (1, 3), (2, 4), (3, 5), (4, 5), (5, 6), (6, 7), (7, 8), (8, 8), (9, 9)] 1.25


In [118]:
s1.shape

(10, 1)